In [25]:
from PIL import Image
import numpy as np
import copy

from bisect import bisect
import matplotlib.pyplot as plt

class mosaic:
    def __init__(self,file):
        print("Carregando a imagem...")
        im = Image.open(file) # Lê a imagem de entrada
        self.width, self.height = im.size
        self.image = np.array(im, dtype="uint8") #  Converte a imagem como array numpy 

    def standardize(self):
        print("Padronizando os canais da imagem...")
        # Tratamento para RGB-GRAYSCALE e/ou JPEG-PNG
        output_np = np.array(np.zeros(shape=(self.height,self.width)), dtype="uint8") # Array numpy de saida
        for i in range(self.height):
            for j in range(self.width):
                aux = round(np.median(self.image[i][j])) # Tira a média dos 3 canais e arredonda o valor 
                output_np[i][j] = aux
        self.image = output_np
        self.original = copy.deepcopy(self.image) # Backup da foto original

    def equal_histogram(self):
        print("Equalizando o histograma...")
        # Cálculo do histograma
        histogram = np.array(np.zeros(shape=256))
        for i in range(self.height): 
            for j in range(self.width):
                histogram[self.image[i][j]] = histogram[self.image[i][j]] + 1

        # Calculando a redistribuição do histograma
        n = self.width*self.height
        equal_histogram_list = []
        print("Level\tIntervalo\tNk\tPr\tDist. Acumulada\tTr(Rk)\tSk")
        cdf = 0
        for index, i in enumerate(range(0, histogram.size, int(histogram.size/8))):	
            val = sum(histogram[elem] for elem in range(i,int(i+histogram.size/8)))
            cdf += val/n
            print(str(index)+"\t["+str(i)+" - "+str(i+histogram.size/8)+"]\t"+str(val)+"\t"+str(val/n)+"\t"+str(cdf)+"\t"+str(cdf*7)+"\t"+str(round(cdf*7)))
            equal_histogram_list.append(round(cdf*7))

        print(equal_histogram_list)

        # Finalmente equalizando o histograma
        for i in reversed(range(0,len(equal_histogram_list))):
            if equal_histogram_list[i] != i:
                print("From: "+str((histogram.size/8)*i)+" - "+str((histogram.size/8)*(i+1))+" To: "+str((histogram.size/8)*equal_histogram_list[i])+" - "+str((histogram.size/8)*(equal_histogram_list[i]+1)))
                for height in range(self.height):
                    for width in range(self.width):
                        if (histogram.size/8)*i <= self.image[height][width] and self.image[height][width] <= (histogram.size/8)*(i+1):
                            level = equal_histogram_list[i]-i
                            self.image[height][width] += (histogram.size/8)*level

    def correct_gama(self,gama):
        print("Corrigindo gama...")
        k = 1
        for i in range(self.height):
            for j in range(self.width):
                self.image[i][j] = pow(self.image[i][j],gama) # Efetua a transformação e garante que o valor do pixel seja arredondado
        if(np.max(self.image) > 255): 
            k = 255/np.max(self.image)

        self.image = np.round(k*self.image)

    def filtro_uniforme(self,L):
        print("Aplicando filtro...")
        window = 2*L+1
        center = L
        filtro = np.zeros((window, window))
        for i in range(window):
            for j in range(window):
                filtro[i][j] = 1/pow(window, 2)

        # Criando a saída e o shadow
        output = np.zeros((self.height+2*center, self.width+2*center))
        output[center:self.height+center,center:self.width+center] = self.image
        altered_input = copy.deepcopy(output)

        for i in range(self.height):
            for j in range(self.width):	
                output[i+center][j+center] = np.sum(altered_input[i:i+window,j:j+window]*filtro)

        self.image = output[center:self.height+center,center:self.width+center] # Descartando o shadow e finalizando a imagem
        
    def interpolation(self, K):
        output = np.zeros((int(self.height*K), int(self.width*K)), "uint8")
        for i in range(0,K*(self.height-1),K):
            for j in range(0,K*(self.width-1),K):
                for i_add in range(K):
                    for j_add in range(K):
                        output[i+i_add][j+j_add] = self.image[int(i/K)][int(j/K)]
        self.image = output

    def downsampling(self, K): 
        output = np.zeros((int(self.height/K), int(self.width/K)), "uint8")
        for i in range(0,self.height, K):
            for j in range(0, self.width, K):
                output[int(i/K)][int(j/K)] = self.image[i][j]
        self.image = output

    def granulate(self, K): # Granulamento por média
        print("Aplicando granulamento...")
        # Caso o valor de granulamento não seja multiplo do tamanho da imagem, temos esse tratamento de borda
        shadow_x = shadow_y = 0
        if(self.height % K != 0):
            shadow_x = K
        if(self.width % K != 0):
            shadow_y = K

        output = np.zeros((self.height, self.width), "uint8")
        for i in range(0,self.height-shadow_x,K):
            for j in range(0,self.width-shadow_y,K):
                aux = round(np.median(self.image[i:i+K, j:j+K]))
                for i_add in range(K):
                    for j_add in range(K):
                        output[i+i_add][j+j_add] = aux
        self.image = output
        
    def recolor(self, boundaries, palette):
        img_out = np.zeros(shape=(self.height, self.width, 3), dtype='uint8')
        for i in range(self.height):
            for j in range(self.width):
                k = bisect(boundaries, self.image[i, j])
                img_out[i, j] = palette[k]
        return img_out

    def generate_image(self):
        self.out = Image.fromarray(self.image, 'L') # Converte o array pra imagem, declarando que a imagem deve estar em tons de cinza
        self.width, self.height = self.out.size
        self.out.show()

In [26]:
boundaries = [16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240]
pallete = [(67, 36, 36),
 (103, 72, 58),
 (106, 74, 63),
 (114, 73, 54),
 (120, 76, 67),
 (122, 52, 47),
 (131, 75, 59),
 (134, 84, 64),
 (139, 89, 66),
 (143, 107, 88),
 (144, 81, 66),
 (151, 84, 63),
 (170, 111, 82),
 (175, 123, 97),
 (182, 124, 98),
 (241, 199, 165)]

In [32]:
obj = mosaic("test#1.png")  
obj.standardize()
obj.equal_histogram()
#obj.correct_gama(0.90)
#obj.filtro_uniforme(3)
obj.granulate(8)

Carregando a imagem...
Padronizando os canais da imagem...
Equalizando o histograma...
Level	Intervalo	Nk	Pr	Dist. Acumulada	Tr(Rk)	Sk
0	[0 - 32.0]	968656.0	0.682032036613	0.682032036613	4.77422425629	5.0
1	[32 - 64.0]	228355.0	0.160785073051	0.842817109664	5.89971976765	6.0
2	[64 - 96.0]	116781.0	0.0822256644957	0.925042774159	6.47529941912	6.0
3	[96 - 128.0]	64089.0	0.0451251540222	0.970167928182	6.79117549727	7.0
4	[128 - 160.0]	28112.0	0.0197936982926	0.989961626474	6.92973138532	7.0
5	[160 - 192.0]	9470.0	0.00666784016898	0.996629466643	6.9764062665	7.0
6	[192 - 224.0]	4049.0	0.00285090653054	0.999480373174	6.99636261222	7.0
7	[224 - 256.0]	738.0	0.000519626826263	1.0	7.0	7.0
[5.0, 6.0, 6.0, 7.0, 7.0, 7.0, 7.0, 7.0]
From: 192.0 - 224.0 To: 224.0 - 256.0
From: 160.0 - 192.0 To: 224.0 - 256.0
From: 128.0 - 160.0 To: 224.0 - 256.0
From: 96.0 - 128.0 To: 224.0 - 256.0
From: 64.0 - 96.0 To: 192.0 - 224.0
From: 32.0 - 64.0 To: 192.0 - 224.0
From: 0.0 - 32.0 To: 160.0 - 192.0
Aplicando g

In [33]:
out = obj.recolor(boundaries, pallete)

In [34]:
Image.fromarray(out, mode='RGB').show()

In [14]:
obj.generate_image()